# LightGCN model RecSys

In [ ]:
# import matplotlib.pyplot as plt
# import networkx as nx
# import numpy as np
# import pandas as pd
import yaml
from lightgcn import LightGCN
from sklearn.model_selection import train_test_split
from utils import *
pd.set_option('display.max_colwidth', None)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device # = "cpu"

In [ ]:
# Load Config
%cd ..
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)
config

# Load Interaction Matrix from csv

In [ ]:
interaction_matrix = pd.read_csv(config['data']['preprocessed'] + "u_i_weight_0.01_0.1_-0.09.csv")
# file 2 -- u_i_weight_0.15_0.35_-0.2.csv

In [ ]:
# mini_im = interaction_matrix.sample(200000)  #100000

### Prepare Train/ Val/ Test dataset

In [ ]:
train_df, test_df = train_test_split(interaction_matrix, test_size=0.1)
# train_df, test_df = train_test_split(mini_im, test_size=0.1)
test_df, val_df = train_test_split(test_df, test_size=0.5)

In [1]:
n_users, n_items, train_df, train_pos_list_df, val_pos_list_df, test_pos_list_df, val_interactions_t, test_interactions_t = prepare_val_test(train_df, val_df, test_df)

NameError: name 'prepare_val_test' is not defined

In [ ]:
edge_index, edge_weight = df_to_graph(train_df, True)

In [ ]:
print("After data pipline")
print("n_users : ", n_users, ", n_items : ", n_items)
print("train_df Size  : ", len(train_df))
print("val_pos_list_df Size : ", len (val_pos_list_df))
print("test_pos_list_df Size : ", len (test_pos_list_df))

In [ ]:
#users, pos_items, neg_items = pos_neg_edge_index(train_pos_list_df, 1, n_users, n_items)
#print("train set size: ", len(users))

In [ ]:
edge_index = edge_index.to(device)
edge_weight = edge_weight.to(device)
interactions_t = interactions_t.to(device)

### Instantiate model and train/val the model

In [ ]:
# Hyper parameters
latent_dim = 80     # aim to 128
n_layers = 3
LR = 0.005
K = 20   # Recall@K
DECAY = 0.0001   # reg loss
BATCH_SIZE = 1024  # train mini batch size

EPOCHS = 50    # total number of epochs
checkpoint_dir = "model-checkpoints"

model = LightGCN(num_nodes=n_users+n_items, embedding_dim=latent_dim, num_layers=n_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
print("Size of Learnable Embedding : ", [x.shape for x in list(model.parameters())])

In [ ]:
bpr_loss, reg_loss, final_loss, recall, precision = train_and_evl(
    n_users, n_items, edge_index, edge_weight, train_pos_list_df, val_pos_list_df, interactions_t, model, optimizer, device, EPOCHS, BATCH_SIZE, K=K, DECAY=DECAY, checkpoint_dir=checkpoint_dir, log_interval=10)

In [ ]:
best_model = torch.load(checkpoint_dir + "/LightGCN_best.pt")
best_epoch = best_model['epoch']
best_val_precision = best_model['precision']
best_val_recall = best_model['recall']

test_model = LightGCN(num_nodes=n_users+n_items, embedding_dim=latent_dim, num_layers=n_layers)
test_model.load_state_dict(best_model['model_state_dict'])

In [ ]:
test_p, test_recall = evaluation(test_model, n_users, n_items, edge_index, edge_weight, test_pos_list_df, interactions_t, K=20)
print(f"Best epoch {best_epoch}")
print(f"Test Precision: {test_p:>0.4f}, Recall: {test_recall:>0.4f}")
print(f"Val Precision: {best_val_precision:>0.4f}, Recall: {best_val_recall:>0.4f}")

In [7]:
import torch, torch_sparse

x = torch.randn(3, 4)
print(x)
s = x.to_sparse()
print(s)
indices = torch.tensor([0, 2])
x1 = torch.index_select(x, 0, indices)
print(x1)
# import pandas as pd
s1 = torch.index_select(s, 0, indices)
print(s1)
print(s1.to_dense())


tensor([[ 1.4358,  1.3681,  0.7431, -0.8240],
        [-0.0645,  0.3822, -0.3929,  0.4640],
        [ 0.2502,  0.1289,  0.4279, -0.0875]])
tensor(indices=tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
                       [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]]),
       values=tensor([ 1.4358,  1.3681,  0.7431, -0.8240, -0.0645,  0.3822,
                      -0.3929,  0.4640,  0.2502,  0.1289,  0.4279, -0.0875]),
       size=(3, 4), nnz=12, layout=torch.sparse_coo)
tensor([[ 1.4358,  1.3681,  0.7431, -0.8240],
        [ 0.2502,  0.1289,  0.4279, -0.0875]])
tensor(indices=tensor([[0, 0, 0, 0, 1, 1, 1, 1],
                       [0, 1, 2, 3, 0, 1, 2, 3]]),
       values=tensor([ 1.4358,  1.3681,  0.7431, -0.8240,  0.2502,  0.1289,
                       0.4279, -0.0875]),
       size=(2, 4), nnz=8, layout=torch.sparse_coo)
tensor([[ 1.4358,  1.3681,  0.7431, -0.8240],
        [ 0.2502,  0.1289,  0.4279, -0.0875]])
